# Laboratorio 8 
    - Francis Aguilar - 22243
    - César López - 22535
    - Gerardo Pineda -22880
    - Angela García -22869 


enlace al repositorio: https://github.com/angelargd8/lab8-deep

# Task 1 - Práctica

In [12]:
import os, numpy as np, pandas as pd
from sklearn.preprocessing import RobustScaler
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, optimizers

## 1. Preparacion de datos

In [13]:
tf.random.set_seed(42)

df = pd.read_csv("data/train.csv")
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values(["store", "item", "date"]).reset_index(drop=True)

df["sales"] = pd.to_numeric(df["sales"], errors="coerce").fillna(0)

def _clip(g):
    lo, hi = g["sales"].quantile([0.01, 0.99]).values
    g["sales"] = g["sales"].clip(lo, hi)
    return g
df = df.groupby(["store","item"], group_keys=False).apply(_clip)


if df["date"].dt.to_period("M").nunique() < df["date"].nunique():
    df["ym"] = df["date"].dt.to_period("M").dt.to_timestamp()
    df = (df.groupby(["store","item","ym"], as_index=False)["sales"].sum()
            .rename(columns={"ym":"date"}))

all_months = pd.DataFrame({"date": pd.date_range(df["date"].min(), df["date"].max(), freq="MS")})
pairs = df[["store","item"]].drop_duplicates()
panel = pairs.merge(all_months, how="cross")
df = (panel.merge(df, on=["store","item","date"], how="left")
           .sort_values(["store","item","date"])
           .reset_index(drop=True))
df["sales"] = df["sales"].fillna(0.0)

df

C:\Users\Francis\AppData\Local\Temp\ipykernel_8764\3001687787.py:13: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(["store","item"], group_keys=False).apply(_clip)


,store,item,date,sales
0,1,1,2013-01-01,331.0
1,1,1,2013-02-01,322.0
2,1,1,2013-03-01,477.0
3,1,1,2013-04-01,522.0
4,1,1,2013-05-01,531.0
...,...,...,...,...
29995,10,50,2017-08-01,2866.0
29996,10,50,2017-09-01,2586.0
29997,10,50,2017-10-01,2507.0
29998,10,50,2017-11-01,2574.0


## 2.Preprocesamiento de datos

In [14]:
HIST = 18   # meses de historia por ventana
HORIZON = 3 # meses a predecir
months = df["date"].sort_values().drop_duplicates().to_numpy()
test_months = months[-HORIZON:]             # últimos 3 meses -> test
val_months  = months[-(HORIZON*2): -HORIZON]      # 3 meses previos -> val
train_months= months[:-(HORIZON*2)]

In [15]:
def make_sequences(group_df, hist=18, horizon=-3, scaler=None):
    g = group_df.sort_values("date").copy()
    sales = g["sales"].values.reshape(-1,1)
    s_sc = scaler.transform(sales)

    X, y, dates_end = [], [], []
    for end in range(hist, len(s_sc) - horizon + 1):
        X.append(s_sc[end-hist:end, :])     
        y.append(s_sc[end:end+horizon, 0])
        dates_end.append(g["date"].iloc[end+horizon-1])
    return np.array(X), np.array(y), np.array(dates_end)

X_train, y_train, X_val, y_val, X_test, y_test = [], [], [], [], [], []
pairs_index_test = []   

for (store, item), g in df.groupby(["store","item"]):
    g = g.sort_values("date").reset_index(drop=True)

    # necesitamos mínimo hist+horizon meses en train para ajustar el scaler de forma robusta
    g_train = g[g["date"].isin(train_months)].copy()
    if len(g_train) < HIST + HORIZON:
        continue

    scaler = RobustScaler(quantile_range=(5,95))
    scaler.fit(g_train[["sales"]].values)

    X_all, y_all, dates_end = make_sequences(g, HIST, HORIZON, scaler)

    for Xi, yi, d_end in zip(X_all, y_all, dates_end):
        if d_end in test_months:
            X_test.append(Xi); y_test.append(yi); pairs_index_test.append((store,item,d_end))
        elif d_end in val_months:
            X_val.append(Xi);  y_val.append(yi)
        elif d_end in train_months:
            X_train.append(Xi); y_train.append(yi)

X_train, y_train = np.array(X_train), np.array(y_train)
X_val,   y_val   = np.array(X_val),   np.array(y_val)
X_test,  y_test  = np.array(X_test),  np.array(y_test)

print("Shapes ->",
      "X_train", X_train.shape, "y_train", y_train.shape,
      "X_val",   X_val.shape,   "y_val",   y_val.shape,
      "X_test",  X_test.shape,  "y_test",  y_test.shape)

Shapes -> X_train (17000, 18, 1) y_train (17000, 3) X_val (1500, 18, 1) y_val (1500, 3) X_test (1500, 18, 1) y_test (1500, 3)


## 3 y 4  Seleccion de modelo y arquitectura

In [16]:
LR = 1e-3
model = models.Sequential([
    layers.Input(shape=(HIST, 1)),
    layers.LSTM(64),
    layers.Dropout(0.2),
    layers.Dense(64, activation="relu"),
    layers.Dropout(0.2),
    layers.Dense(HORIZON)
])
model.compile(
    loss="mse",
    optimizer=optimizers.Adam(learning_rate=LR),
    metrics=[tf.keras.metrics.MeanAbsoluteError(name="mae")]
)
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                   │ (None, 64)             │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,251 (83.01 KB)

 Trainable params: 21,251 (83.01 KB)

 Non-trainable params: 0 (0.00 B)

## 5. Entrenamiento con EarlyStopping

In [17]:
BATCH = 256
EPOCHS = 200

In [18]:
es  = callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
rlr = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-5)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH,
    callbacks=[es, rlr],
    verbose=1
)

Epoch 1/200
67/67 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 0.0486 - mae: 0.1738 - val_loss: 0.0131 - val_mae: 0.0874 - learning_rate: 0.0010
Epoch 2/200
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0160 - mae: 0.1006 - val_loss: 0.0118 - val_mae: 0.0838 - learning_rate: 0.0010
Epoch 3/200
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0134 - mae: 0.0920 - val_loss: 0.0094 - val_mae: 0.0776 - learning_rate: 0.0010
Epoch 4/200
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0120 - mae: 0.0871 - val_loss: 0.0084 - val_mae: 0.0724 - learning_rate: 0.0010
Epoch 5/200
67/67 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 0.0110 - mae: 0.0830 - val_loss: 0.0074 - val_mae: 0.0688 - learning_rate: 0.0010
Epoch 6/200
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0103 - mae: 0.0799 - val_loss: 0.0073 - val_mae: 0.0679 - learning_rate: 0.0010
Epoch 7/200
67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0098 - mae: 0.0777 - val_loss: 0.0065 - val_mae: 0.0657 - learning_rate: 0.0010
Epoch 

## 6. Evaluación del modelo 

In [19]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Evaluación directa con Keras
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=1)
print(f"Loss (MSE): {test_loss:.4f}")
print(f"MAE: {test_mae:.4f}")

# Predicciones sobre el conjunto de test
y_pred = model.predict(X_test)

# Cálculo de métricas adicionales
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("\nMétricas de evaluación:")
print(f"MAE  = {mae:.4f}")
print(f"MSE  = {mse:.4f}")
print(f"RMSE = {rmse:.4f}")


47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0027 - mae: 0.0404
Loss (MSE): 0.0027
MAE: 0.0404
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step

Métricas de evaluación:
MAE  = 0.0404
MSE  = 0.0027
RMSE = 0.0521


## 7. Ajuste de hiperparámetros

# Task 2 - Teoría

### 1. ¿Cuál es el problema del gradiente de fuga en las redes LSTM y cómo afecta la efectividad de LSTM para el pronóstico de series temporales? 
El problema del gradiente de fuga en las redes LSTM, es que aunque las LSTM están diseñadas para mitigarlo no son inmunes, ya que en LSTM el estado de la celda va de esta manera:

$$
c_t = f_t \odot c_{t-1} + i_t \odot g_t
$$

Y en la retropropagación a través del tiempo hace que la gradiente hacia:
$$
c_{t-1}
$$

se multiplique por el producto de las compuertas del olvido, entonces el resultado del producto tiende a 0 cuando la secuencia es larga o las compuertas se cierran más, entonces el gradiente se desvanece y si fuera mayor que 1 explotaría. Y eso pasa en la práctuca ya que se puede dar la saturación de sigmoides, sesgo del olvido mal iniciado, secuencias largas y normalización o escala pobre de entradas. Y esto afecta al pronóstico, porque no capta dependencias de largo plazo, tiene predicciones que sub-reaccionan a las señales antiguas, tiene una convergencia lenta y es inestable, ya que a varios pasos el error crece y el modelo olvida el contexto útil.


### 2. ¿Cómo se aborda la estacionalidad en los datos de series temporales cuando se utilizan LSTM para realizar pronósticos y qué papel juega la diferenciación en el proceso? 

El manejo de estacionalidad en LSTM se puede modelar la estacionalidad como caracteristicas o al quitarla y recomponerla. Ya que, la diferenciación lo que hacec es estabilizar y facilitar el aprendizaje, pero requiere de reintegrar y puede sobreactuar si la estacionalidad no es rígida. En el caso que la estacionalidad sea fuerte o regular es mejor deseasonalizarla, quitarla al entrenar el LSTM sobre la serie limpua y reaplicar al estacionalidad al final. Mientras que si es informativa para el modelo, es mejor agregarlas como features exógenas.



### 3. ¿Cuál es el concepto de "tamaño de ventana" en el pronóstico de series temporales con LSTM y cómo afecta la elección del tamaño de ventana a la capacidad del modelo para capturar patrones a corto y largo plazo? 

El concepto de tamaño de ventana o tambien llamado lookback o W, es cuantos pasos del pasado se le entrega al modelo para predecir el futuro. Y el W si es una ventana corta o pequeña, tiende a patrones de corto plazo, entonces este aprende rápidamente señales locales, pero no ve ciclos largos. Mientras que si es una ventana larga o grande, tiende a patrones de largo plazo, estos pueden capturar tendencias y estacionalidades largas si están presentes en la ventana, tiene parámetros más efectivos, mayor varianza/overfitting, y el entrenamiento es más lengo. Por el desvanecimiento de gradientes si la información útil está muy lejos el LSTM no puede aprovechar todo. Por lo tanto para evaluar el tamaño de la ventana y saber como ajustarla es imporante analizar la serie y su ciclo. Si la H es grande, los datos limitados para evitar W demasiado grandes y si tiene múltiples ciclos. 




